In [ ]:
import jax.numpy as jnp
from scipy import signal
import numpy as np
import time

from jaxdsp.processors import fir_filter, iir_filter, clip, delay_line, lowpass_feedback_comb_filter as lbcf, allpass_filter, freeverb, serial_processors
from jaxdsp.training import train, evaluate, process

### TODO
* rename git:khiner/jax_audio_experiments to git:khiner/python_audio_processors with a `/processors/jax` directory
* copy all (non-jax) audio processors in git:khiner/jupyter_notebooks here
* publish `python_audio_processors` as a public library
* delete all (now duplicated) processors in git:khiner/jupyter_notebooks, replace with `python_audio_processors` imports
* move all existing analysis into a new `/analysis` directory here

In [ ]:
from IPython.display import Audio
from scipy.io.wavfile import read as readwav

# sample_rate = 44_100
# X = np.random.randn(sample_rate * 40)
sample_rate, X = readwav('../audio/speech-male.wav')
Audio(X, rate=sample_rate)
tail_length = 24 * sample_rate # let it ring out
X = np.concatenate([X, np.zeros(tail_length)])

In [ ]:
processor = freeverb
presets = processor.PRESETS
audio_for_preset = {preset_name: processor.tick_buffer({'params': params, 'state': processor.init_state()}, X)[1] for preset_name, params in presets.items()}

In [ ]:
# Avoid a harsh clip at the end of the sample.
def apply_release(X, release_samples=int(0.2*sample_rate)):
    return X * np.concatenate([np.ones(X.shape[-1] - release_samples), np.linspace(1.0, 0.0, release_samples)])

output_for_preset = {preset_name: Audio(apply_release(audio.T), rate=sample_rate) for preset_name, audio in audio_for_preset.items()}

In [ ]:
output_for_preset['flat_space']

In [ ]:
output_for_preset['expanding_space']

## TODO
* demonstrate single-sample tick
* plot processing time normalized to real_time = 1.0
* implement streaming processing (chunked)
  - Okay, I think this is the path:
    * Client web application uses Web Audio API to stream microphone to WebRTC peer ([example](https://webrtc.github.io/samples/src/content/peerconnection/webaudio-input/))
    * Set up a locally-running Flask server listening to a WebRTC stream ([example](https://github.com/aiortc/aiortc/blob/main/examples/server/server.py))
    * Process the incoming audio chunks, then send them back to the stream
  - Example matplotlib charts with streaming audio from microphone: https://github.com/markjay4k/Audio-Spectrum-Analyzer-in-Python
  - This paper does what I'm trying to do: https://www.researchgate.net/publication/326885247_Real-Time_Digital_Signal_Processing_Using_pyaudio_helper_and_the_ipywidgets
  - And this is what's behind that paper: https://github.com/mwickert/scikit-dsp-comm
  - I think the above is too heavy-weight in terms of dependencies, and I'm working through broken deps.
  - https://dvcs.w3.org/hg/audio/raw-file/tip/webaudio/webrtc-integration.html
    * Keywords: WebRTC integration with the Web Audio API
* compare to C++ performance
* provide real-time control over parameters via js
  - just in jupyter for now
* perform inference real-time, displaying estimated parameter values via auto-controlled, disabled knobs
  - interface idea: two separate reverb controllers, on top and bottom of screen, each with a knob for each param. You can mute or solo either reverb separately. At any point, you can hit the 'learn' button for either reverb. This will disable its knobs, and will then listening to the input & output audio of the other (still user-controllable) reverb and live-update its own parameters to try and match the other reverb parameter values.
* check if URL is available: python_audio_processors.io
  - otherwise, just use karlhiner.com/python_audio_processors
* port to ^
* charts for impulse response, magnitude spectrogram and phase, updating in real-time